<a href="https://colab.research.google.com/github/francji1/01RAD/blob/main/R/01RAD_Ex11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 01RAD Exercise 11

* Robust Regression - How to deal with Outliers and Leverages
* Very fast intro of Robust regression - M, MM, LTS, LWS


In [ ]:
######################
######################
# get requirements for this exercise (and previous ones)
list_of_packages <- c("tidyverse", "car", "lattice", "MASS","graphics","psych","effects","corrplot","ISLR","robustbase","rrcov","robust","quantreg")
                     # "caret","tidymodels","GGally","nortest","lmtest","leaps","ISLR","robustbase","rrcov","robust","quantreg"
missing_packages <- list_of_packages[!(list_of_packages %in% installed.packages()[,"Package"])]
if(length(missing_packages)) install.packages(missing_packages)
lapply(list_of_packages, library, character.only = TRUE)

######################
# check our settings
#getwd()
#print(.libPaths())
print(sessionInfo())
#print(version)

# delete graphics
#dev.off()
#par(mfrow = c(1, 1))
############################################################
############################################################


In [ ]:

# Dataset from the last exercise
# Use Hertzsprung-Russell Diagram Data of Star Cluster CYG OB1
? starsCYG
summary(starsCYG)
attach(starsCYG)
#png('starsCYG.png')
plot(log.light ~ log.Te,xlim=rev(range(log.Te)+c(-0.1,0.1)),ylim = c(3.5,6.5), pch=20, col = "red3", xaxs="i",yaxs="i",
     main="Hertzsprung-Russell Diagram Data",xlab="SURFICE TEMPERATURE", ylab="LUMINIOUS OUTPUT")
#dev.off()



In [ ]:
#png('starsCYG.png')
plot(log.light ~ log.Te,xlim=rev(range(log.Te)+c(-0.1,0.1)),ylim = c(3.5,6.5), pch=20, col = "red3", xaxs="i",yaxs="i",
     main="Hertzsprung-Russell Diagram Data",xlab="SURFICE TEMPERATURE", ylab="LUMINIOUS OUTPUT")
#dev.off()

In [ ]:
# Simple linear regression by OLS 
# Let's take Luminosity as a response variable

OLS_stars <-(lm(log.light ~ log.Te,data=starsCYG))
summary(OLS_stars)
summary(influence.measures(OLS_stars))
out_index<-as.numeric(row.names(summary(influence.measures(OLS_stars))))
out_index


In [ ]:
round(sort(hatvalues(OLS_stars),decreasing = T)[1:10],4)
dim(starsCYG)
r = sum(hatvalues(OLS_stars)) # == 2 no. of regress. coefficients
n = 47
sum(hatvalues(OLS_stars) > 3*r/n)
starsCYG[hatvalues(OLS_stars) > 3*r/n,]

In [ ]:
round(sort(dffits(OLS_stars),decreasing = T)[1:10],4)
starsCYG[abs(dffits(OLS_stars)) > 3*sqrt(r/(n-r)),]

In [ ]:
options(repr.plot.width=18, repr.plot.height=14)
op <- par(mfrow=c(2,3))
plot(OLS_stars,which=1)
plot(OLS_stars,which=2)
plot(OLS_stars,which=3)
plot(OLS_stars,which=4)
plot(OLS_stars,which=5)
plot(OLS_stars,which=6)
par(op)



In [ ]:
plot(log.light ~ log.Te,xlim=rev(range(log.Te)+c(-0.1,0.1)),ylim = c(3.5,6.5), pch=20, col = "red3", xaxs="i",yaxs="i",
     main="Hertzsprung-Russell Diagram Data",xlab="SURFICE TEMPERATURE", ylab="LUMINIOUS OUTPUT")
abline(OLS_stars)
points(log.Te[out_index],log.light[out_index],pch = 22, col = "blue", lwd =2)

In [ ]:
c(1:length(log.Te))[log.Te<3.7]
sort(OLS_stars$residuals)
out_index2 <- c(1:length(log.Te))[log.Te<3.9]


In [ ]:
OLS_stars2 <- lm(log.light[-out_index] ~ log.Te[-out_index])
opar<-par(mfrow = c(2,2))
plot(OLS_stars2)
par(opar)
summary(OLS_stars2)


In [ ]:

OLS_stars3 <- lm(log.light[-out_index2] ~ log.Te[-out_index2])
opar<-par(mfrow = c(2,2))
plot(OLS_stars3)
par(opar)
summary(OLS_stars3)


In [ ]:

plot(log.light ~ log.Te,xlim=rev(range(log.Te)+c(-0.1,0.1)),ylim = c(3.5,6.5), pch=20, col = "red3", xaxs="i",yaxs="i",
     main="Hertzsprung-Russell Diagram Data",xlab="SURFICE TEMPERATURE", ylab="LUMINIOUS OUTPUT")
points(log.Te[log.Te<3.7],log.light[log.Te<3.7],pch = 22, col = "blue", lwd =2)
points(log.Te[7],log.light[7],pch = 22, col = "cyan", lwd =2)
abline(OLS_stars2, col ="blue")
abline(OLS_stars3,col ="cyan")
legend("bottomright",cex=0.7,legend = c("OLS without 4pts","OLS without 5pts"),
       lty = c(1,1),col = c("blue","cyan"))



##  Direct Robust approach

Robust regression by M-estimator
 `rlm(formula, data,..., method = c("M","MM"))` 
 * M estimate with diferent types of weight functions psi (derivative of rho) 
 * MM estimate - S estimate + M estimate 


See Definitions of Psi-Functions Available in Robustbase
on https://cran.r-project.org/web/packages/robustbase/index.html


In [ ]:
# Huber family of functions using tuning parameter k = 1.345.
names(.Mpsi.tuning.defaults)
plot(huberPsi, ylim=c(-1.4, 5), leg.loc="topright",cex=0.8)


In [ ]:
# Bisquare family functions using tuning parameter k = 4.685
x = seq(-5,5,by=0.01)
source(system.file("xtraR/plot-psiFun.R", package = "robustbase", mustWork=TRUE))
p.psiFun( x, "biweight", par = 4.685)


In [ ]:
# GGW (Generalized Gauss-Weight function )family of functions 
# using tuning parameters a = 1.387, b = 1.5 and c = 1.063.
p.psiFun(x, "GGW", par = c(-.5, 1, .95, NA))


In [ ]:
# rlm from MASS
# Summary plot for starsCYG
M_huber_stars    <- rlm(log.light ~ log.Te, method="M", psi = psi.huber)    # Huber
M_hampel_stars   <- rlm(log.light ~ log.Te, method="M", psi = psi.hampel)   # Hampel
M_bisquare_stars <- rlm(log.light ~ log.Te, method="M", psi = psi.bisquare) # Tukey
MM_bisquare_stars<- rlm(log.light ~ log.Te, method="MM",psi = psi.bisquare) # Tukey
MM_ggw_stars     <- rlm(log.light ~ log.Te, method="MM",psi = psi.ggw)      # GGW
plot(starsCYG,main="Hertzsprung-Russell Diagram Data",xlab="SURFICE TEMPERATURE", ylab="LUMINIOUS OUTPUT")
abline(M_huber_stars, col="coral",lwd=1.8)
abline(M_hampel_stars, col="blue")
abline(M_bisquare_stars, col="cyan")
abline(MM_bisquare_stars, col="firebrick", lwd = 2)
abline(MM_ggw_stars, col="green")
legend("bottomleft",cex=0.7,legend = c("M Huber","M Hampel","M Tukey","MM Tukey","MM GGW"),
       lty = c(1,1,1,1,1),col = c("coral","blue","cyan","firebrick","green"))



In [ ]:
coef(summary(M_huber_stars))
coef(summary(M_hampel_stars))
coef(summary(M_bisquare_stars))
coef(summary(MM_bisquare_stars))
coef(summary(MM_ggw_stars))

summary(M_huber_stars)

# alternative to MM from rlm is lmrob from robustbase package
# lmrob computes fast MM-type estimator
# ? mlrob - MM-type Estimators for Linear Regression
# ? nlrob - Robust Fitting of Nonlinear Regression Models


In [ ]:
MM_stars2 <- lmrob(log.light ~ log.Te, init = "S")
MM_stars2$coefficients
summary(MM_stars2)
coef(summary(MM_stars2))

# Least Absolute Residuals / L1 Regression
# LAR_stars2 <- lmrob.lar(log.Te,log.light)
# LAR_stars2$coefficients

All of the previous estimators are not high breakdown point. First high breakdown point estimators are LMS and LTS:
https://web.ipac.caltech.edu/staff/fmasci/home/astro_refs/LeastMedianOfSquares.pdf

In [ ]:
# Robust regression by LMS-estimator
LMS_stars  <- lmsreg(log.light ~ log.Te)
LMS_stars



The objective function for TLS is
$$ S(\beta )=\sum _{j=1}^{h}r_{(j)}(\beta )^{2} $$
where for fixed $\beta$, $r_{(j)}(\beta )$ denotes the $i$th ordered absolute values of the residual. 


$$ \beta^{(TLS,h)} = argmin_{\beta}  \sum_{j=1}^{h}r_{(j)}(\beta )^{2} $$

In [ ]:
# Robust regression by LTS-estimator
# two possibilities - ltsreg from MASS (lqs(formula, data, ..., method = c("lts")))
#                   - ltsReg from robustbase
?ltsreg
?ltsReg


In [ ]:
LTS_stars  <- ltsreg(log.light ~ log.Te)

LTS_stars  <- ltsReg(log.light ~ log.Te, alpha=0.9)
# when stop the algorithm?
LTS_stars  <- ltsReg(log.light ~ log.Te, alpha=0.5, nsamp="best")
LTS_stars  <- ltsReg(log.light ~ log.Te, alpha=0.5, nsamp="best")
LTS_stars  <- ltsReg(log.light ~ log.Te, alpha=0.5, nsamp="exact")




In [ ]:
# Summary plot for starsCYG
OLS_stars  <- lm(log.light ~ log.Te)
MM_stars   <- rlm(log.light ~ log.Te,method="MM")
LTS_stars  <- ltsreg(log.light ~ log.Te)
LTS2_stars <- ltsReg(log.light ~ log.Te, alpha=0.9)
LMS_stars  <- lmsreg(log.light ~ log.Te)
plot(starsCYG,main="Hertzsprung-Russell Diagram Data",xlab="SURFICE TEMPERATURE", ylab="LUMINIOUS OUTPUT")
abline(OLS_stars, col="black")
abline(MM_stars, col="blue")
abline(LTS_stars, col="red")
abline(LTS2_stars, col="darkred")
abline(LMS_stars, col="green")
legend("bottomleft",cex=0.7,legend = c("OLS","MM","LTS, h=0.5","LTS h=0.9","LMS"),
         lty = c(1,1,1,1,1),col = c("black","blue","red","darkred","green"))

### Summary 

Robust Location and Scatter Estimation via  Minimum Covariance Determinant
https://arxiv.org/pdf/1709.07045.pdf
`mcd_0.5  <- covMcd(starsCYG)`

`mcd_0.75 <-  covMcd(starsCYG, alpha = 0.75)`

GM-estimation
  * MASS: rlm() with method=''M'' (Huber, Tukey, Hampel), Choice for the scale estimator: MAD, Huber Proposal 2

S-estimation
  * robust: lmRob with estim=''Initial''
  * robustbase: lmrob.S

MM-estimation
  * MASS: rlm() with method=''MM''
  * robust: lmRob (with estim='''Final'', default)
  * robustbase: lmrob()


In [ ]:
# Problems with general position
data(exAM)
? exAM
summary(exAM)


In [ ]:
#Because the points are not in general position, both LMS and LTS typically fail
summary(lm1   <- lm(y ~ x, data=exAM))
summary(rlm0  <- rlm(y ~ x, data=exAM,method="MM"))
summary(rlm1  <- ltsReg(y ~ x, data=exAM))
summary(rlm1b <- ltsReg(y ~ x, alpha = 0.9,data=exAM))
summary(rlm2  <- lmsreg(y ~ x, data=exAM))


In [ ]:
plot(exAM,main="Antille and May artificial data set")
abline(lm1, col="black")
abline(rlm0, col="blue")
abline(rlm1, col="red")
abline(rlm1b, col="darkred")
abline(rlm2, col="green")
legend("topright",cex=0.7,legend = c("OLS","MM","LTS, h=0.5","LTS h=0.9","LMS"),
       lty = c(1,1,1,1,1),col = c("black","blue","red","darkred","green"))



In [ ]:
# THE PROBLEM IS HOW LARGE h (alpha) WE SHOULD SELECT FOR LTS
# for example in Stars data - nearly now change
# How trimmed parameter h (alpha) change the estimation?
LTS_stars09 <- ltsReg(log.light ~ log.Te, alpha=0.9)
LTS_stars08 <- ltsReg(log.light ~ log.Te, alpha=0.8,nsamp="exact",adjust=F,mcd = F)
LTS_stars07 <- ltsReg(log.light ~ log.Te, alpha=0.7)
LTS_stars06 <- ltsReg(log.light ~ log.Te, alpha=0.6)
LTS_stars05 <- ltsReg(log.light ~ log.Te, alpha=0.5,nsamp="exact",adjust=F,mcd = F)
plot(starsCYG,main="Hertzsprung-Russell Diagram Data",xlab="SURFICE TEMPERATURE", ylab="LUMINIOUS OUTPUT")
abline(LTS_stars09, col="black", lwd = 2)
abline(LTS_stars08, col="blue", lwd = 7)
abline(LTS_stars07, col="red", lwd = 5)
abline(LTS_stars06, col="darkred", lwd = 3)
abline(LTS_stars05, col="green", lwd = 1)
legend("bottomleft",cex=0.7,legend = c("LTS h=0.9","LTS h=0.8","LTS h=0.7","LTS h=0.6","LTS h=0.5"),
       lty = c(1,1,1,1,1),col = c("black","blue","red","darkred","green"))


In [ ]:
summary(LTS_stars05)
summary(LTS_stars08)


In [ ]:

# Let's check outputs
LTS_stars05$quan
LTS_stars08$quan

LTS_stars05$crit
LTS_stars08$crit


In [ ]:

# But coefficents are same :(
LTS_stars05$coefficients
LTS_stars08$coefficients


In [ ]:

# h points with smallest resids
LTS_stars05$best
LTS_stars08$best


In [ ]:

# Apply ordinary LS to h best (selected) points
LTS_stars05OLS <- lm(log.light ~ log.Te, data=starsCYG[LTS_stars05$best,])
LTS_stars06OLS <- lm(log.light ~ log.Te, data=starsCYG[LTS_stars06$best,])
LTS_stars07OLS <- lm(log.light ~ log.Te, data=starsCYG[LTS_stars07$best,])
LTS_stars08OLS <- lm(log.light ~ log.Te, data=starsCYG[LTS_stars08$best,])
LTS_stars09OLS <- lm(log.light ~ log.Te, data=starsCYG[LTS_stars09$best,])


In [ ]:

# commpare
summary(LTS_stars05OLS)
summary(LTS_stars08OLS)


In [ ]:

plot(starsCYG,main="Hertzsprung-Russell Diagram Data",xlab="SURFICE TEMPERATURE", ylab="LUMINIOUS OUTPUT")
abline(LTS_stars09OLS, col="black", lwd = 2)
abline(LTS_stars08OLS, col="blue", lwd = 2)
abline(LTS_stars07OLS, col="red", lwd = 2)
abline(LTS_stars06OLS, col="darkred", lwd = 2)
abline(LTS_stars05OLS, col="green", lwd = 2)
legend("bottomleft",cex=0.7,legend = c("LTS h=0.9","LTS h=0.8","LTS h=0.7","LTS h=0.6","LTS h=0.5"),
       lty = c(1,1,1,1,1),col = c("black","blue","red","darkred","green"))





In [ ]:

ggplot(data=starsCYG, aes(log.Te, log.light)) +                    # use starsCYGdata frame
  geom_point() +                                                  
  geom_smooth(method="lm", aes(colour="OLS"), se=F) +          
  geom_smooth(method="rlm", aes(colour="M"), se=T) +          
  stat_smooth(method=function(formula,data,weights=weight) rlm(formula,
                                                               data,
                                                               weights=weight,
                                                               method="MM",
                                                               psi=psi.bisquare),
              fullrange=TRUE,aes(colour="MM bisquare"))+
  labs(colour=NULL)  



# Nacteme si funkce potrebne pro vypocet LWS


In [ ]:

# Nagenerujeme si data 
n=100
p=3
beta0 <- matrix(c(2,3,-4),p,1)


In [ ]:

set.seed(123)
X1 <- rep(1,times=n)
X2 <- rnorm(n,10,2)
X3 <- rexp(n,1/5)
X  <- cbind(X1, X2, X3)
e  <- rnorm(n,0,1)
Y  <- X%*%beta0 + e


In [ ]:

# Vytvorime 5% outlieru
Yoi    = sample(n)[1:ceiling(n/20)]
Y[Yoi] = 5*Y[Yoi]


In [ ]:

# Vytvorime 5% Leverage pointu
Xoi     = sample(n)[1:ceiling(n/20)]
X[Xoi,2:p]  = 0.2*X[Xoi,2:p]  



In [ ]:
LWS <- function(y, x, w, niter){
    
    n <- nrow(x)
    p <- ncol(x)
    
   rWr_LWS=Inf;
   # iterations = rep(0,times=n);
   # Outer Cycle
   for(i in 1:niter) {
   # Performing one iteration in the outer cycle
   #    [b_OneIter,S_OneIter,rWr_OneIter,perm_OneIter,iterations_OneIter]=OneIter(X,Z,Y,w);
   OneIter = LWS_OneIter(y,x,w)  
   
    if (rWr_LWS > OneIter$rWr){
    b_LWS      =  OneIter$b
    rWr_LWS    =  OneIter$rWr
    perm_LWS   =  OneIter$perm
    }
   }
    
    result <- list(beta=b_LWS, resid = rWr_LWS, rank=perm_LWS)
    return(result)
}

LWS2 <- function(y, x, hp, gp, niter){
  n <- nrow(x)
  p <- ncol(x)
  w <- Weights2(n,hp,gp)
    
  rWr_LWS=Inf;
  # iterations = rep(0,times=n);
  # Outer Cycle
  for(i in 1:niter) {
    # Performing one iteration in the outer cycle
    #    [b_OneIter,S_OneIter,rWr_OneIter,perm_OneIter,iterations_OneIter]=OneIter(X,Z,Y,w);
    OneIter = LWS_OneIter(y,x,w)  
    if (rWr_LWS > OneIter$rWr){
      b_LWS      =  OneIter$b
      rWr_LWS    =  OneIter$rWr
      perm_LWS   =  OneIter$perm
      }
  }
    
  result <- list(beta=b_LWS, resid = rWr_LWS, rank=perm_LWS)
  return(result)
}

LWS_OneIter = function(Y, X, w){
    n <- nrow(X)
    p <- ncol(X)
    W <- diag(w);
    tau         <- 0.00001;
    rWr_OneIter <- Inf;
    b_OneIter   <- matrix( rep(0,times=p),p,1);

    # Select the initial values
    index  = sample(n)
    SX     = X[(index[1:p+2]),]
    SY     = Y[index[1:p+2]]
    while(kappa(SX) > 1e9 ){
        index  = sample(n)
        SX     = X[(index[1:(p+5)]),]
        SY     = Y[index[1:(p+5)]]
    }
    #print(kappa(SX))
    b = OLS_qr(SY,SX) 

    # Run the inner cycle
    while (sum(abs(b_OneIter - b) < tau) != p){
          #  rWr_OneIter > rWr && sum(abs(b_OneIter - b) < tau) ~= p;
          b_OneIter     = b

          r      = (Y-X%*%b)^2
          rI     = order(r)
          sX     = X[rI,]
          sY     = Y[rI,]  
          
          swX    = (sqrt(w)*sX)[which(w != 0),]
          swY    = (sqrt(w)*sY)[which(w != 0)]
          b      = OLS_qr(swY,swX) 
    }
    b_OneIter      = b
    r_OneIter      = (Y-X%*%b_OneIter)^2
    perm_OneIter   = order(r_OneIter)
    rWr_OneIter    = (r_OneIter[perm_OneIter]^2)%*%w   
    # S      = t((sY-sX%*%b)^2)%*%w
    result_OneIter <- list(b=b_OneIter, rWr = rWr_OneIter, perm=perm_OneIter)
    return(result_OneIter)
} 


OLS_qr = function(y, x, tol=1.e-07 ){
    qr_x <- qr(x, tol=tol, LAPACK=TRUE)
    b<-qr.coef(qr_x, y)
    return(b)
} 

OLS_lm <- function (y, x){
    b<-coef(lm(y ~ x - 1))
    return(b)
} 

OLS_def <- function (y, x) {
    b <- as.vector((solve(t(x)%*%x))%*%(t(x)%*%y))
    return(b)
} 

Weights1 <- function(n){
      # Generating weights
      # h      - number of trimmed observations (observations with zero weights)
      # g      - number of observations with weights equal to 1
      # h-g-1  - number of observations with rapidly decreasing weights
  h <- ceiling(0.9*n)
  g <- ceiling(0.8*n)
  w <- rep(0,times=n)
  w[1:g] <- 1
  w[g:h] <- seq(1,0,length=h-g+1)
  return(w)
}

Weights2 <- function(n,hp,gp){
      # Generating weights
      # h      - number of trimmed observations (observations with zero weights)
      # g      - number of observations with weights equal to 1
      # h-g-1  - number of observations with rapidly decreasing weights
  h <- ceiling(hp*n)
  g <- ceiling(gp*n)
  w <- rep(0,times=n)
  w[1:g] <- seq(1,0.9,length=g)
  w[g:h] <- seq(0.9,0,length=h-g+1)
  return(w)
}

In [ ]:
w = Weights1(n)
OLS_qr(Y,X)
LWS(Y,X,w,100)$beta
summary(ltsReg(Y ~ X2+X3, alpha=0.9))$coeff



In [ ]:

#Use Hertzsprung-Russell Diagram Data of Star Cluster CYG OB1
? starsCYG
summary(starsCYG)
attach(starsCYG)


In [ ]:

w_star       <- Weights1(nrow(starsCYG))
YY = as.matrix(log.light)
XX = cbind(rep(1,times=nrow(YY)), log.Te)
LWS_stars_lm <- LWS(YY, XX, w_star, 1000)
LTS_stars    <- ltsReg(log.light ~ log.Te, alpha=0.9)
OLS_stars    <- lm(log.light ~ log.Te)


In [ ]:

plot(starsCYG,main="Hertzsprung-Russell Diagram Data",xlab="SURFICE TEMPERATURE", ylab="LUMINIOUS OUTPUT")
abline(OLS_stars, col="black")
abline(LWS_stars_lm$beta, col="blueviolet")
abline(LTS_stars, col="red")
legend("bottomleft",cex=0.7,legend = c("OLS","LTS, h=0.9","LWS h=0.9"),
       lty = c(1,1,1),col = c("black","blueviolet","red"))



In [ ]:

#POZN: Pokud muzete vyvarujte se pocitani LS pomoci primeho vypoctu inverzi matice z X'T !!!
#      Pouzivejte NUMERICKY robustnejsi pristupy.

##################
# Task 01:
###################
#1)
# Vyzkousejte si LWS na datech z minule hodiny, 
# porovnejte vysledky s dalsimi metodami, zkuste zmenit vstupni parametry


#2)
# Vytvorte funkci generujici vahy pro LWS Weights2  tak, 
#  aby pro prvnich g pozorovani klesala konstannte tak, ze na celkovem intervalu (0,g) klesne o 0.1 
#  aby na inervalu (g,h) klesla z 0.9 na 0
#  aby pro (n-h) pozorovani byla rovna 0 (nulova na intervalu (h,n))
#  g a h byly vstupni parametry

# Provedte experiment, kde otestujete konzistenci odhadu regresnich parametru pomoci metody LWS
# Pro pocet pozorovani 50,100,200,300,400,500 nagenerujte data, kde regresni matice X bude mit
#  1. sloupec intercept
#  2. sloupec pozorovani z binomickeho rozdeleni s parametry k=10, p = 0.3 posunute o +1
#  3. sloupec pozorovani z normalniho rozdeleni N(10,2)
#  4. sloupec pozorovani z normalniho rozdeleni N(20,3)
#  5. sloupec pozorovani z exponencialniho rozdeleni se stredni hodnotou 8 
# vektor beta0 volte napriklad beta0=c(2,3,-2,2,3)

# Pro dany pocet pozorovani si 1000 vygenerujte nova data, pro ne spoctete odhad pomoci OLS, LWS a LTS
# Pro kazdy odhad parametru spoctete empiricky median z SE (popripade MSE)
# vykreslete zavislost medianu (popripade eMSE) na poctu pozorovani pro vsechny 3 odhady

#  Experiment opakujte s 0%, 5% a 10% odlehlych pozorovani, ktera muzete vytvorit podle sveho uvazeni 
########################


In [ ]:

#########################
# Ukazka reseni Ukolu 01
ni    = c(50,100,200,300,400,500,1000)
nrep  = 50
niter = 100


In [ ]:

p=5
beta0 = matrix(c(2,3,-2,2,3),5,1) 


In [ ]:

betaMSE_OLS <- matrix(NA,length(ni),p)    
betaMSE_LWS <- matrix(NA,length(ni),p) 
betaMSE_LTS <- matrix(NA,length(ni),p) 


In [ ]:

betaMedSE_OLS <- matrix(NA,length(ni),p)    
betaMedSE_LWS <- matrix(NA,length(ni),p) 
betaMedSE_LTS <- matrix(NA,length(ni),p) 


In [ ]:

for (i in 1:length(ni)){
  betaSE_OLS <- matrix(NA,nrep,p)    
  betaSE_LWS <- matrix(NA,nrep,p) 
  betaSE_LTS <- matrix(NA,nrep,p) 
  for (j in 1:nrep){
    n  <- ni[i]
    X1 <- rep(1,times=n)
    X2 <- rbinom(n,10,0.3)+1
    X3 <- rnorm(n,10,2)
    X4 <- rnorm(n,20,3)
    X5 <- rexp(n,1/8)
    X  <- cbind(X1, X2, X3, X4, X5)
    e  <- rnorm(n,0,1)
    Y  <- X%*%beta0 + e
    
    # Vytvorime 5% outlieru
    Yoi    = sample(n)[1:ceiling(n/20)]
    Y[Yoi] = 5*Y[Yoi]
    
    # Vytvorime 5% Leverage pointu
    Xoi     = sample(n)[1:ceiling(n/20)]
    X[Xoi,2:p]  = 0.2*X[Xoi,2:p]  
    
    betaSE_OLS[j,] <- OLS_qr(Y,X)
    betaSE_LWS[j,] <- LWS2(Y,X,0.9,0.8,niter)$beta
    betaSE_LTS[j,] <- ltsReg(Y ~ X2+X3+X4+X5, alpha=0.9)$coeff
  }
  # odecist beta0
  betaMSE_OLS[i,] <- colMeans((betaSE_OLS - t(matrix(rep(beta0, times = nrep),nrow=p)))^2)   
  betaMSE_LWS[i,] <- colMeans((betaSE_LWS - t(matrix(rep(beta0, times = nrep),nrow=p)))^2) 
  betaMSE_LTS[i,] <- colMeans((betaSE_LTS - t(matrix(rep(beta0, times = nrep),nrow=p)))^2) 
  
  betaMedSE_OLS[i,] <- colMedians((betaSE_OLS - t(matrix(rep(beta0, times = nrep),nrow=p)))^2)  
  betaMedSE_LWS[i,] <- colMedians((betaSE_LWS - t(matrix(rep(beta0, times = nrep),nrow=p)))^2) 
  betaMedSE_LTS[i,] <- colMedians((betaSE_LTS - t(matrix(rep(beta0, times = nrep),nrow=p)))^2) 
}


In [ ]:

#par(mfrow = c(2, 2))
#plot(ni,betaMSE_OLS[,2], log="y", ylim=c(0.01, 20),yaxt="n", type = "l", col="black")
for (ii in 1:5){
  par(mfrow = c(1, 1))
  plot(seq(50,5000,by=1), 1/seq(50,5000,by=1),type = "l", col="black",log="y", ylim=c(0.0001, 1))
  lines(ni,betaMSE_LWS[,ii],type = "l", col="red")
  lines(ni,betaMSE_LTS[,ii],type = "l", col="blue")
}
################################


In [ ]:

#####################
######## Today Exercise

# Investiagate following datasets
# Try to find influence points (leverages, outliers)
# Compare robust estiamtors with different input parameters with
# classical OLS method with removed influence points
# plot regression lines of more estimators together and discuss resluts


# Crime data
# Source: Statistical Methods for Social Sciences
# Alan Agresti and Barbara Finlay (1997)
crimedata = read.csv("data/crimedata.csv",sep=",")
# Variables
#state id (sid)
#state name (state)
#violent crimes per 100,000 people (crime)
#murders per 1,000,000 (murder)
#the percent of the population living in metropolitan areas (pctmetro)
#the percent of the population that is white (pctwhite)
#percent of population with a high school education or above (pcths)
#percent of population living under poverty line (poverty)
#percent of population that are single parents (single).
summary(crimedata)
pairs.panels(crimedata[-51,4:10],smooth=F,lm=T,scale=T,pch=20)

# 0) Explain crime by single
# 1) Explain crime by poverty and single
# 2) Explain crime by poverty, single, pcths, pctwhite, pctmetro
# Extra) If you want, do the same for murders

